In [1]:
# import BigchainDB and create an object
from bigchaindb_driver import BigchainDB
bdb_root_url = 'http://192.168.1.108:9984'
bdb = BigchainDB(bdb_root_url)

# generate a keypair
from bigchaindb_driver.crypto import generate_keypair
alice, bob = generate_keypair(), generate_keypair()

# create a digital asset for Alice
game_boy_token = {
    'data': {
        'token_for': {
            'game_boy': {
                'serial_number': 'LR35902'
            }
        },
        'description': 'Time share token. Each token equals one hour of usage.',
    },
}

# prepare the transaction with the digital asset and issue 10 tokens for Bob
prepared_token_tx = bdb.transactions.prepare(
    operation='CREATE',
    signers=alice.public_key,
    recipients=[([bob.public_key], 10)],
    asset=game_boy_token)

# fulfill and send the transaction
fulfilled_token_tx = bdb.transactions.fulfill(
    prepared_token_tx,
    private_keys=alice.private_key)
bdb.transactions.send_commit(fulfilled_token_tx)

# Use the tokens
# create the output and inout for the transaction
transfer_asset = {'id': fulfilled_token_tx['id']}
output_index = 0
output = fulfilled_token_tx['outputs'][output_index]
transfer_input = {'fulfillment': output['condition']['details'],
                  'fulfills': {'output_index': output_index,
                               'transaction_id': transfer_asset['id']},
                  'owners_before': output['public_keys']}

# prepare the transaction and use 3 tokens
prepared_transfer_tx = bdb.transactions.prepare(
    operation='TRANSFER',
    asset=transfer_asset,
    inputs=transfer_input,
    recipients=[([alice.public_key], 3), ([bob.public_key], 7)])

# fulfill and send the transaction
fulfilled_transfer_tx = bdb.transactions.fulfill(
    prepared_transfer_tx,
    private_keys=bob.private_key)
sent_transfer_tx = bdb.transactions.send_commit(fulfilled_transfer_tx)

print(sent_transfer_tx)

print(sent_transfer_tx==fulfilled_transfer_tx)

ReadTimeout: HTTPConnectionPool(host='192.168.1.108', port=9984): Read timed out. (read timeout=20)

2019年3月10日17点24分鬼知道我遭遇了什么，http no s；使用最新的driver，老驱动安装的时候会有一堆依赖装不上，docker imag bigchaindb真是心机，只有all-in-one这个版本的run命令，最新的一点文档都没有。。。不是人民币玩家就这枚被摧残。

以前写的脚本好i想调用了老包，被更改了，座椅history只有历史意义了，又要从头开始了。

In [12]:
# 查询
response = bdb.assets.get(search='f62973f97ea406b91198f74db49f4000090357d56e8745e8b2997977b5eb0028')
print(response)

[{'data': {'token_for': {'game_boy': {'serial_number': 'LR35902'}}, 'description': 'Time share token. Each token equals one hour of usage.'}, 'id': 'f62973f97ea406b91198f74db49f4000090357d56e8745e8b2997977b5eb0028'}]
